In [1]:
import math
import tsplib95
import statistics
import numpy as np
import pandas as pd
from itertools import groupby
from scipy.special import _logsumexp

In [2]:
def first_index(arr: list, idx: float) -> float:
    if (len(arr) == 1):
        return arr
    else:
        return arr[0]

### Normal functions of the algorithm

In [3]:
def herding(route: list, Vt: list, fit: list, n: int, dimension: int, acc: list, time: list) -> list:
    
    route_copy = np.array(route)
    fit1 = np.sort(fit, 0, kind="heap")
    idx = np.argsort(fit, 0, kind="heap")
    route1 = np.zeros(len(route))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))

    for i in range(n):
        route1[i] = first_index(route_copy[idx[i]], idx[i])
        Vt1[i, :] = Vt[idx[i][0], :]
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return route1, Vt1, fit1, acc1, time1

In [4]:
def update_v(Vt: list, n: int, dimension: int, acc: list, time: list, route: list, fit: list, eye: int) -> list:
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = np.copy(Vt)
    r = len(acc)
    l = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[l-1]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[1] + fit[2]) / 2
    f = 0
    tempg = 0
    temps = 0

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(n):
        for j in range(dimension):
            #Velocity updation of dogs
            if (i <= 2):
                Vt1[i][j] = math.sqrt(np.power(Vt[i][j], 2) + (2 * acc[i][j]) * abs(route[i]))
                
            #Velocity updation of sheep
            if (i > 2):
                if eye == 1:
                    Vt1[i][j] = math.sqrt(np.power(Vt1[f][j], 2) + (2*acc2[f][j]) * abs(route[i]))

                else:
                    #Velocity updation of gathered sheep
                    if ((fit1 - fit[i]) > (fit2 - fit[i])):
                        Vt1[i][j] = math.sqrt(np.power(Vt1[0,j], 2) + (2*acc[0,j]))
                        #tempg[i] = i        
                
                    #Velocity updation of stalked sheep
                    if ((fit1 - fit[i]) <= (fit2 - fit[i])):
                        Vt1[i][j] = math.sqrt(np.power(Vt[right_dog][j]*math.tan(np.random.randint(1,89)), 2) + (2*acc[right_dog][j]*abs(route[right_dog]))) + math.sqrt(np.power(Vt[left_dog][j]*math.tan(np.random.randint(91,179)), 2) + (2*acc[left_dog][j]*abs(route[left_dog])))
                        Vt1[i][j] = (Vt1[i][j]) / 2
                        #temps[i] = i
    
    #Updation of time and acceleration
    for i in range(n):
        s = 0
        for j in range(dimension):
            acc1[i][j] = abs((Vt1[i][j]) - (Vt[i][j])) / (time[i][j])
            s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]
            
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [5]:
def check(pop: list, n: int, dimension: int, acc: list, Vt: list, time: list, objf: object) -> list:
    pop1 = np.copy(pop)
    acc1 = np.copy(acc)
    time1 = np.copy(time)
    Vt1 = np.copy(Vt)
    for i in range(n):
        for j in range(dimension):
            if (pop[i][j] >= np.max(objf) or pop[i][j] <= np.min(objf) or pop[i][j] == 0):
                pop1[i][j] = np.random.random() * (np.max(objf) - np.min(objf)) + np.max(objf)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(n):
        for j in range(dimension):
            if (math.isnan(acc[i][j]) == 1 or acc[i][j] == 0):
                pop1[i][j] = np.random.random() * (np.max(objf) - np.min(objf)) + np.max(objf)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(Vt[i][j]) == 1 or Vt[i][j] == 0):
                pop1[i][j] = np.random.random() * (np.max(objf) - np.min(objf)) + np.max(objf)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                pop1[i][j] = np.random.random() * (np.max(objf) - np.min(objf)) + np.max(objf)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()


    return pop1, acc1, time1, Vt1

### Discretization of the problem

In [15]:
def discrete_fitness(route, n, objf):
    fit1 = np.zeros((len(route), 1))

    for i in range(len(route)-1):
        if (i != -1):    
            fit1[i] = objf[int(route[i])][int(route[i+1])]
    #print("Fit -> ", fit1, "\n")
    
    #best fit and position
    best_fit = np.min(fit1)
    pos = np.argmin(fit1)
    
    return fit1, best_fit, pos 

In [7]:
def discrete_generate(n: int, dimension: int) -> list:
    boundary_no = 1
    pop = np.zeros((n, dimension))

    if boundary_no == 1:
        pop = np.random.randint(0, 1+1, size=(n,dimension))

    x = np.random.random(size = (n, dimension))
    
    return pop, x

In [8]:
def discrete_update(Vt: list, time: list, acc: list, n: int, dimension: int, eye: int) -> list:
    pop1 = np.zeros((n, dimension))
    for i in range(n):
        for j in range(dimension):
            #Updating the position of dogs
            if (i <= 2):
                pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
                #print("Pop1 -> ", pop1[i][j],"\n")
                #print("Discrete update dogs-> ", discretizing(Vt[i][j], np.nanmax(Vt[i])) * discretizing(time[i], np.nanmax(time)) + (1/2) * discretizing(acc[i][j], np.nanmax(acc[i])) * (np.power(discretizing(time[i], np.nanmax(time)), 2)),"\n")
                
            #Updating position of sheep
            if (i > 2):
                if eye == 1:
                    pop1[i][j] = Vt[i][j] * time[i] - (1/2) * acc[i][j] * (np.power(time[i], 2))
                    #print("Pop1 -> ", pop1[i][j],"\n")
                    #print("Discrete update sheep 1 -> ", discretizing(Vt[i][j], np.nanmax(Vt[i])) * discretizing(time[i], np.nanmax(time)) - (1/2) * discretizing(acc[i][j], np.nanmax(acc[i])) * (np.power(discretizing(time[i], np.nanmax(time)), 2)),"\n")
                
                else:
                    pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
                    #print("Pop1 -> ", pop1[i][j],"\n")
                    #print("Discrete update sheep 2 -> ", discretizing(Vt[i][j], np.nanmax(Vt[i])) * discretizing(time[i], np.nanmax(time)) + (1/2) * discretizing(acc[i][j], np.nanmax(acc[i])) * (np.power(discretizing(time[i], np.nanmax(time)), 2)),"\n")
                
    return pop1

### Funtions to define  the route

In [12]:
# The first city is 0
# Need to put at least one sheep per city
# Create a function to make the reward strategy

def city_choice(n: int) -> list:
    route = []
    city = []
    sheep = np.random.randint(0, n)

    for map in range(n):
        city.append(map)
    
    while len(city) != 0:
        if sheep not in city:
            sheep = np.random.randint(0, n)
        else:
            route.append(sheep)
            city.remove(sheep)

    return route

# Create a random number each iteration and after that made a line of the matrix receive that number
def generate_city(n: int, dim: int) -> list:
    city_matrix = np.zeros((n, dim))
    sheep = city_choice(n)

    for i in range(n):
        for j in range(dim):
            if (sheep[i] == j):
                city_matrix[i][j] = 1
    
    x = np.random.random(size = (n, dim))
    
    return city_matrix, x

# Debug function
# There's no need to use this function in the main algorithm, just in the test file
def print_position_of_sheep(city_matrix: list):
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] != 0):
                print("Position -> ",j)

# Function to create route
def create_route(city_matrix: list) -> list:
    route = []
    for i in range(len(city_matrix)):
        for j in range(len(city_matrix)):
            if (city_matrix[i][j] == 1):
                route.append(j)
    print("Route -> ",route,"\n")

    return route

# Function to verify route
def verify_route(route: list) -> bool:
    # Append each position in a list, after that verify if the route is valid in a different ways
    valid = True
    for i in range(len(route)):
        if (i == (-1) and route[i] != 0):
            valid = False
    valid = same_cities(route)

    return valid

def same_cities(route: list) -> bool:
    valid = True
    aux = route[-1]
    i = 1
    j = 1
    while i != (len(route) - 2):
        if (aux == route[j]):
            valid = False
            break
        j += 1
        if (j == (len(route) - 1)):
            aux = route[i]
            i += 1
            j = 1

    return valid    

def punishment(city_matrix: list, route: list, weight: int) -> list:
    aux_verify = verify_route(route)
    aux_route = same_cities(route)
    position = route[1]
    if (aux_route == True or aux_verify == False):
        city_matrix[0][int(position)] *= np.power(weight, 2)
    
    # print("Route -> ",route,"\n")
    # print("City matrix -> ",city_matrix,"\n")
    
    return city_matrix

### Application of TSP

In [13]:
problem = tsplib95.load('../data/domain/bays29.tsp')

In [16]:
#Maximum no. of iterations
gen = 200

#Population size
n = problem.dimension

#Optimization function name
dim = problem.dimension

#Intialize the population(init_p-Population,acc-acceleration of each individual)
init_p, acc = generate_city(n, dim)

#Vt = velocity of each individuals
Vt = np.zeros((n, dim))

#Time of each individual
time = np.random.random(size = (n,1))

#Max fitness value
fopt = math.inf
#Variable to store fitness
fit = np.zeros((n, 1))
pop = init_p
#print_position_of_sheep(pop)
#k = counter variable for iterations required for Eyeing mechanism
k = 1
fopt_1 = np.zeros((gen, 1))
# Generate a route and at the same time verify if this route is valid
# Needs to generate before the loop and adjust in till the iterations turns it valid
route = create_route(pop)

# Weight that's gonna be used in the punishment function
weight = np.max(problem.dimension)

for g in range(gen):
    valid = verify_route(route)
    # Calculate fitness of indivuals
    # Need to remember to debug this function
    fit, maxf, pos = discrete_fitness(route, n, getattr(problem, 'edge_weights'))
    #print("Valid -> ", valid,"\n")
    eye = 0
    if g == 0:
        fopt = maxf
    
    #Finding the optimum fitness value
    if fopt > maxf:
        fopt = maxf
    
    fopt_1[g] = fopt
    if g > 0:
        if fopt_1[g] > fopt_1[g-1]:
            k += 1
            if k > 5:
                eye = 1
                k = 0
    
    route, Vt, fit, acc, time = herding(route, Vt, fit, n, dim, acc, time)
    
    Vt, acc, time, r1, l1, tempg, temps = update_v(Vt, n, dim, acc, time, route, fit, eye)

    pop = discrete_update(Vt, time, acc, n, dim, eye)
    # Checking the range of the population is maintained
    pop, acc, time, Vt = check(pop, n, dim, acc, Vt, time, getattr(problem, 'edge_weights'))
    #fit = punishment(pop, route, weight)
    print("Route -> ",route,"\n")

Route ->  [9, 16, 28, 19, 0, 8, 2, 21, 13, 1, 12, 25, 11, 22, 3, 7, 18, 20, 15, 17, 4, 27, 6, 5, 24, 14, 26, 23, 10] 

Fit ->  [[193.]
 [319.]
 [135.]
 [150.]
 [152.]
 [217.]
 [435.]
 [ 44.]
 [234.]
 [101.]
 [208.]
 [169.]
 [310.]
 [239.]
 [192.]
 [177.]
 [167.]
 [161.]
 [178.]
 [287.]
 [121.]
 [314.]
 [386.]
 [301.]
 [125.]
 [177.]
 [ 41.]
 [215.]
 [  0.]] 

Route ->  [10. 26. 21.  1.  4. 24. 28. 19.  0. 20. 18. 25. 14.  7. 15.  3.  9. 12.
 23.  8. 13. 22. 17.  5. 11. 27. 16.  6.  2.] 

Fit ->  [[216.]
 [290.]
 [278.]
 [ 88.]
 [307.]
 [350.]
 [135.]
 [150.]
 [ 65.]
 [167.]
 [283.]
 [280.]
 [233.]
 [ 98.]
 [121.]
 [ 42.]
 [ 57.]
 [ 74.]
 [221.]
 [368.]
 [309.]
 [296.]
 [296.]
 [ 55.]
 [112.]
 [380.]
 [313.]
 [509.]
 [  0.]] 

Route ->  [ 2.  3.  5.  9.  0. 12.  1.  7. 11. 15. 28. 19. 20. 10. 23. 14. 21. 25.
 18. 26. 17. 22.  4. 13. 16. 24.  8. 27.  6.] 

Fit ->  [[374.]
 [234.]
 [167.]
 [157.]
 [113.]
 [101.]
 [183.]
 [193.]
 [279.]
 [275.]
 [135.]
 [ 88.]
 [254.]
 [215.]
 [176.]
 [113

C:\Users\artur\AppData\Local\Temp/ipykernel_27544/3957245458.py:60: RuntimeWarning: invalid value encountered in double_scalars
  s += (_logsumexp.logsumexp(Vt1[i][j]) - _logsumexp.logsumexp(Vt[i][j])) / acc1[i][j]


Route ->  [13.  0.  1. 24. 25.  5. 14. 18. 27. 23.  7.  9. 20. 15. 12. 22.  3. 28.
 21. 10. 19. 26. 17.  6. 16.  8.  4.  2. 11.] 

Fit ->  [[297.]
 [107.]
 [251.]
 [352.]
 [118.]
 [272.]
 [ 56.]
 [213.]
 [ 93.]
 [ 57.]
 [188.]
 [ 95.]
 [161.]
 [ 75.]
 [184.]
 [239.]
 [205.]
 [332.]
 [103.]
 [187.]
 [189.]
 [233.]
 [266.]
 [313.]
 [360.]
 [ 46.]
 [171.]
 [312.]
 [  0.]] 

Route ->  [11.  8. 14. 23. 15. 27.  9. 21.  0. 25. 20.  4. 12. 10.  7. 19.  3. 18.
 26. 22.  1. 17.  5. 13.  2.  6. 28. 24. 16.] 

Fit ->  [[ 89.]
 [286.]
 [176.]
 [ 62.]
 [155.]
 [189.]
 [193.]
 [341.]
 [169.]
 [110.]
 [ 65.]
 [163.]
 [176.]
 [272.]
 [216.]
 [ 70.]
 [ 68.]
 [121.]
 [ 80.]
 [271.]
 [199.]
 [296.]
 [331.]
 [391.]
 [509.]
 [435.]
 [350.]
 [228.]
 [  0.]] 

Route ->  [16. 23. 20.  3. 19. 26. 11. 25. 18. 15.  4.  0. 14. 12. 27.  9.  1.  7.
 24. 22. 10.  8. 17.  5. 21. 28. 13.  6.  2.] 

Fit ->  [[287.]
 [104.]
 [137.]
 [ 70.]
 [189.]
 [241.]
 [169.]
 [283.]
 [ 66.]
 [232.]
 [124.]
 [228.]
 [117.]
 [158.]
 

KeyboardInterrupt: 